# Text Features

In [84]:
# General libraries.
import re
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
#from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

In [85]:
train_df = pd.read_json("../input/train.json")
test_df = pd.read_json("../input/test.json")

In [86]:
y=train_df['interest_level']
y2 = []
for k in y:
    if k == 'low': 
        y2.append(1)
    elif k == 'medium': 
        y2.append(2)
    elif k == 'high':
        y2.append(3)
    else: 
        y2.append(-1) 

y2=pd.DataFrame(y2)
#change so they are ordered 

X = train_df.drop('interest_level', 1)

from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(
    X, y2, test_size=0.33, random_state=1)

X_train.head()
#print(y[0:3], y2[0:3], y_train.head())

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
93524,1.0,2,a01c99eb2cfdde327e1691e17d6696ba,2016-04-28 05:42:04,"29TH! PRIME KIPS BAY, HIGH FLOOR, LARGE FLEX 2 BEDROOM IN LUX DM BLG WITH A TERRACE AND VIEWSMidrise postwar building stands 17 stories and features a 24-hour doorman, garden, laundry room, valet services and a rooftop deck. Located in the heart of Kips Bay, there is the convenience of being close to all the supermarkets, bars, restaurants, movie theaters, nightlife, transportation and more. As you enter this large FLEX 2 apartment, the living room has two exposures (North and West) which allow for tons of sunlight to come in to the apartment all throughout the day. The thermopane windows help control the flow of hot/cold air coming in to the apartment. In addition, the bedroom is able to accommodate a king-size bed and you have access to a small terrace. The pass through kitchen is clean and modern with an electric stove, dishwasher, microwave and granite counter tops. The bathroom have also been renovated with Kohler fixtures and ceramic tile floors. *low fee<br /><br />To view f...",East 29th Street,"[Roof Deck, Doorman, Elevator, Laundry in Building, Laundry in Unit, Dishwasher, Hardwood Floors, No Fee]",40.7412,6937971,-73.9772,2a1ee03b449700c3a15dd8c9a505c525,"[https://photos.renthop.com/2/6937971_4419d62ee3384c96c8b0eabc35abef12.jpg, https://photos.renthop.com/2/6937971_cede41e1d04b9d99e230b3de5db4fe79.jpg, https://photos.renthop.com/2/6937971_f7f0b0cbfbfdf2dd6e0ddc1f1e85e982.jpg, https://photos.renthop.com/2/6937971_15fff0709c54359f5b312ee651488473.jpg, https://photos.renthop.com/2/6937971_7e9df16bff58b396c57a29f47049838b.jpg, https://photos.renthop.com/2/6937971_9c18ff5064d4027dba9500b673057373.jpg, https://photos.renthop.com/2/6937971_23103deb56fe5494393494f41160608d.jpg]",2850,340 East 29th Street
116815,2.0,3,0,2016-04-06 01:34:28,Very large 3-bedroom apartment in an elevator building. Three blocks from the 1/A/C trains and four to Columbia Presbyterian Hospital and Columbia College of Physicians and Surgeons.,West 171st Street,"[Dogs Allowed, Cats Allowed]",40.8433,6828255,-73.9397,1b431b2ef64b0b5ca12dfdf471770e93,[],2800,620 West 171st Street
3545,1.0,2,a1e5cd46adc406e4abb33a74042e9b37,2016-06-11 03:52:07,2BR,111 Mott St,"[No Fee, Cats Allowed]",40.7176,7140748,-73.9975,0b4c2bdea168a99e824ae44dbac348f5,[https://photos.renthop.com/2/7140748_65f66bc0c021de6ec97085269160c0e8.jpg],2750,111 MOTT ST
52853,1.0,1,0,2016-05-29 01:12:35,Beautiful walk-up building 6th Floor aptEast 65th Street just off of First AveQuiet tree lined blockAmazing apartment features a nice living roomOpen kitchenDishwasher!Microwave!Bedroom fits a FULL bedCharming bathroomGorgeous hardwood floorsBeaming natural lightPerfect share apartment Steps to F train at 63rd and Lexington or6 train at 68th St and Lexington.M66 Crosstown bus 67th and York.,East 65th Street,[Pre-War],40.7630,7080836,-73.9577,d24abc331d8b47f73a369795690c010a,"[https://photos.renthop.com/2/7080836_42011311a073bb9e7e7483b8e3374414.jpg, https://photos.renthop.com/2/7080836_fcd3bbf5566ee212156c7804575919ee.jpg, https://photos.renthop.com/2/7080836_8977d2439fbf11b52d263c2b473dcdb9.jpg, https://photos.renthop.com/2/7080836_b76aefe26884170b44bbc9f41921ce00.jpg, https://photos.renthop.com/2/7080836_358121bcc2a77ae26f08bbfa9928e090.jpg, https://photos.renthop.com/2/7080836_c7046e311aa96599fa3a9d44f1041d5a.jpg]",1995,429 East 65th Street
20234,1.0,3,f2ba3c3a3c29443417d42d4816911c75,2016-06-01 06:15:40,"This fantastic newly renovated convertible 3 bedroom apartment has tons of space and natural light. Kitchen offers black granite counter tops and all stainless steel appliances. Marble tile bath with brand new shower.<br /><br />Luxury building located in Midtown East, near Grand Central Station offers its residents amenities such as a doorman, an on-site spa, a laundry room, an

In [87]:
from string import punctuation
def txt_features(dat):
    strlen = [len(x) for x in (dat['description'])]
    numwords = [len(x.split()) for x in (dat['description'])]
    numcaps = [sum(1 for c in x if c.upper()) for x in (dat['description'])]
    numpunct = [sum(1 for c in x if c in punctuation) for x in (dat['description'])]
    richness = [len(set(x)) / (len(x)+0.001) for x in dat['description']]

    ddf = pd.DataFrame()
    ddf['strlen'] = strlen
    ddf['numwords'] = numwords
    ddf['numcaps'] = numcaps
    ddf['numpunct'] = numpunct
    ddf['richness'] = richness
    return ddf

In [88]:
from sklearn.ensemble import RandomForestClassifier
 
ddf_tr = txt_features(X_train)
ddf_de = txt_features(X_dev)

rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1) #-1 means use all available cores
rfc.fit(ddf_tr, y_train)
print('Training Score: %.2f%%' % (rfc.score(ddf_tr, y_train) * 100))

#rfc.fit(ddf_de, y_dev) #oops, this would be cheating
print('Dev Score: %.2f%%' % (rfc.score(ddf_de, y_dev) * 100))


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Training Score: 93.59%
Dev Score: 66.47%


In [89]:
importances = rfc.feature_importances_
features = ddf_tr.columns

sort_indices = np.argsort(importances)[::-1]
sorted_features = []
for i in sort_indices:
    sorted_features.append(features[i])

vals = importances[sort_indices]
labels = sorted_features

print labels
print vals

['richness', 'numwords', 'strlen', 'numcaps', 'numpunct']
[ 0.25899405  0.19026255  0.18841789  0.18759741  0.1747281 ]


## Let's take a look at num photos vs scores

In [90]:
pd.options.display.max_colwidth #50
pd.options.display.max_colwidth = 1000

num_photos = [len(x) for x in (X_train['photos'].values)]
dd_tp = pd.DataFrame()
dd_tp['num_photos'] = num_photos


num_photos_dev = [len(x) for x in (X_dev['photos'].values)]
dd_dp = pd.DataFrame()
dd_dp['num_photos'] = num_photos_dev

lr = LogisticRegression()
lr.fit(dd_tp, y_train)

print('\t Training Accuracy: %.2f%% \t Test Accuracy: %.2f%%' % (lr.score(dd_tp, y_train)*100, 
                                                                 lr.score(dd_dp, y_dev)*100))


	 Training Accuracy: 69.60% 	 Test Accuracy: 69.19%


## Lets add numphotos to our text analysis test

In [98]:
ddf_tr['num_photos'] = num_photos
ddf_de['num_photos'] = num_photos_dev

rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1) #-1 means use all available cores
rfc.fit(ddf_tr, y_train)
print('\nRF\tTraining Score:%.2f%% \
       \tDev Score: %.2f%%' % (rfc.score(ddf_tr, y_train) * 100, rfc.score(ddf_de, y_dev) * 100))


importances = rfc.feature_importances_
features = ddf_tr.columns

sort_indices = np.argsort(importances)[::-1]
sorted_features = []
for i in sort_indices:
    sorted_features.append(features[i])

vals = importances[sort_indices]
labels = sorted_features

lr = LogisticRegression()
lr.fit(ddf_tr, y_train)
print('LR \t Training Accuracy: %.2f%% \t Dev Accuracy: %.2f%%' % (lr.score(ddf_tr, y_train)*100, 
                                                                 lr.score(ddf_de, y_dev)*100))



print labels
print vals

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RF	Training Score:94.15%        	Dev Score: 67.10%
LR 	 Training Accuracy: 69.53% 	 Dev Accuracy: 69.15%
['richness', 'numcaps', 'strlen', 'numwords', 'numpunct', 'num_photos']
[ 0.21859142  0.17003016  0.16858574  0.1677809   0.16266951  0.11234227]


## What about comparing addresses

In [112]:
sa_len = [len(sa) for sa in X_train['street_address']]
da_len = [len(da) for da in X_train['display_address']]
dif_len = np.subtract(sa_len,da_len)

dd_t = pd.DataFrame()
dd_t['dif_len'] = dif_len

sa_len_dev = [len(sa) for sa in X_dev['street_address']]
da_len_dev = [len(da) for da in X_dev['display_address']]
dif_len_dev = np.subtract(sa_len_dev,da_len_dev)


dd_d = pd.DataFrame()
dd_d['dif_len'] = dif_len_dev


lr = LogisticRegression()
lr.fit(dd_t, y_train)

print('LR \t Training Accuracy: %.2f%% \t Dev Accuracy: %.2f%%' % (lr.score(dd_t, y_train)*100, 
                                                                 lr.score(dd_d, y_dev)*100))


LR 	 Training Accuracy: 69.58% 	 Dev Accuracy: 69.23%


add in address dif to bigger set

In [114]:
ddf_tr['addr_diff'] = dif_len
ddf_de['addr_diff'] = dif_len_dev

rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1) #-1 means use all available cores
rfc.fit(ddf_tr, y_train)
print('\nRF\tTraining Score:%.2f%% \
       \tDev Score: %.2f%%' % (rfc.score(ddf_tr, y_train) * 100, rfc.score(ddf_de, y_dev) * 100))

lr = LogisticRegression()
lr.fit(ddf_tr, y_train)
print('LR \t Training Accuracy: %.2f%% \t Dev Accuracy: %.2f%%' % (lr.score(ddf_tr, y_train)*100, 
                                                                 lr.score(ddf_de, y_dev)*100))

importances = rfc.feature_importances_
features = ddf_tr.columns

sort_indices = np.argsort(importances)[::-1]
sorted_features = []
for i in sort_indices:
    sorted_features.append(features[i])

vals = importances[sort_indices]
labels = sorted_features

print labels
print vals

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RF	Training Score:94.50%        	Dev Score: 67.47%
LR 	 Training Accuracy: 69.51% 	 Dev Accuracy: 69.17%
['richness', 'strlen', 'numcaps', 'numwords', 'numpunct', 'num_photos', 'addr_diff']
[ 0.19915102  0.1601959   0.15910764  0.15526029  0.14743492  0.11128729
  0.06756294]
